In [1]:
from github_analysis.github.client.client import GitHubClient

await GitHubClient.users.get_pull_requests(
    username="kamaal111", filter_states=["MERGED"]
)

{'user': {'pullRequests': {'nodes': [{'createdAt': '2019-07-22T13:44:23Z',
     'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-22T13:42:45Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-22T13:41:50Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-13T09:58:31Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-06T19:16:18Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-06T13:11:01Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-06T12:06:40Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-06T10:03:45Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-05T14:03:38Z', 'author': {'login': 'kamaal111'}},
    {'createdAt': '2019-07-05T12:20:39Z', 'author': {'login': 'kamaal111'}}],
   'pageInfo': {'endCursor': 'Y3Vyc29yOnYyOpK5MjAxOS0wNy0wNVQxNDoyMDozOSswMjowMM4RkyU4',
    'hasNextPage': False,
    'hasPreviousPage': True,
    'startCursor': 'Y3Vyc29yOnYyOpK5MjAxOS0wNy0